In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
# devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
# testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [4]:
print(testSentences[3])

[['Die', 'O'], ['These', 'O'], [',', 'O'], ['Schlatter', 'B-PER'], ['sei', 'O'], ['Antisemit', 'O'], ['gewesen', 'O'], [',', 'O'], ['wurde', 'O'], ['seither', 'O'], ['in', 'O'], ['der', 'O'], ['theologischen', 'O'], ['Fachliteratur', 'O'], ['nicht', 'O'], ['mehr', 'O'], ['vertreten', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [6]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'I-PER': 16, 'PADDING_TOKEN': 0, 'I-LOCderiv': 1, 'B-OTHpart': 3, 'B-LOC': 4, 'B-ORG': 18, 'I-OTH': 9, 'I-LOC': 5, 'I-PERpart': 11, 'I-OTHderiv': 6, 'I-LOCpart': 17, 'B-ORGpart': 7, 'I-ORG': 8, 'B-PERpart': 10, 'B-LOCderiv': 19, 'I-ORGderiv': 20, 'I-ORGpart': 21, 'O': 22, 'B-OTH': 12, 'B-LOCpart': 13, 'I-PERderiv': 23, 'B-PERderiv': 2, 'B-PER': 14, 'B-ORGderiv': 15, 'B-OTHderiv': 24, 'I-OTHpart': 25}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'allUpper': 3, 'PADDING_TOKEN': 0, 'allLower': 2, 'initialUpper': 4, 'other': 5, 'mainly_numeric': 6, 'numeric': 1, 'contains_digit': 7}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'β': 5, ']': 6, '·': 8, '„': 276, 'ế': 9, '€': 10, 'n': 12, 'Á': 13, 'ς': 318, 's': 15, '士': 71, 'ô': 7, 'έ': 18, 'í': 19, 'x': 20, 'ą': 22, 'é': 23, '冲': 306, 'š': 24, 'т': 25, '¹': 28, '(': 70, 'O': 323, 'ệ': 11, '“': 29, '算': 30, 'œ': 33, '\x94': 324, 'ν': 303, 'α': 88, '→': 196, '별': 228, '0': 34, 'з': 35, 'ż': 280, 'N': 36, 'ñ': 37, 'Ü': 40, 'ß': 224, '術': 42, 'с': 43, 'ç': 44, 'о': 45, '\x80': 46, 'R': 47, '¤': 21, 'ά': 50, 'У': 52, ',': 53, '?': 56, '»': 57, 'å': 59, 'τ': 62, 'Å': 61, 'ḳ': 63, '’': 262, 'À': 229, '…': 180, 'г': 66, 't': 67, 'в': 68, 'Ş': 72, '학': 76, 'Þ': 332, '懿': 75, 'ي': 74, 'W': 127, 'П': 278, 'Ä': 77, '別': 65, 'B': 78, 'Ż': 270, '\x95': 132, 'Т': 82, 'Z': 81, 'и': 321, 'ض': 83, 'a': 38, '\x99': 27, '殿': 231, 'п': 86, 'D': 87, '3': 258, 'Л': 90, 'e': 79, 'ı': 92, '5': 93, '▪': 94, 'X': 39, 'î': 98, 'ā': 96, 'µ': 97, '東': 99, '³': 26, '«': 100, 'è': 102, 'ب': 107, 'σ': 103, 'н': 104, 'Е': 106, 'л': 108, '°': 109, '</S>': 2, 'ǒ': 234, 'λ': 111, '‘': 112, 'Æ':

In [14]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


# Test Model

In [ ]:
# import importlib
# importlib.reload(models)

In [ ]:
tmp_model_filename = 'model_lstm2_germeval.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_2lstm()

In [ ]:
#import importlib
#importlib.reload(utils)
# print(models.max_sequence_length)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm2_germeval.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm2_germeval.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_2lstm()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=256), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, None, 52, 100 33200       char_embedding[0][0]             
__________________________________________________________________________________________________
word

Epoch 1/10
1499/1500 [============================>.] - ETA: 0s - loss: 0.0525 - acc: 0.9836
New maximum F1 score: 0.7672692673644147 (before: 0) Saving to model_lstm2_germeval.1.h5
1500/1500 [==============================] - 909s 606ms/step - loss: 0.0525 - acc: 0.9836 - val_loss: 0.0096 - val_acc: 0.9892
Epoch 2/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0122 - acc: 0.9889
New maximum F1 score: 0.7890669180018849 (before: 0.7672692673644147) Saving to model_lstm2_germeval.1.h5
1500/1500 [==============================] - 893s 595ms/step - loss: -0.0122 - acc: 0.9889 - val_loss: -0.0400 - val_acc: 0.9902
Epoch 3/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0655 - acc: 0.9901
New maximum F1 score: 0.8006700167504186 (before: 0.7890669180018849) Saving to model_lstm2_germeval.1.h5
1500/1500 [==============================] - 882s 588ms/step - loss: -0.0656 - acc: 0.9901 - val_loss: -0.0927 - val_acc: 0.9900
Epoch 4/10
1499/1500 [===========

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close